In [3]:
# Standard Imports
import numpy as np 
import pandas as pd
from math import log

In [4]:
df = pd.read_csv('iris.csv')
df.head(10)

,Slen,Swid,Plen,Pwid,Class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
5,4.6,3.4,1.4,0.3,Iris-setosa
6,5.0,3.4,1.5,0.2,Iris-setosa
7,4.4,2.9,1.4,0.2,Iris-setosa
8,4.9,3.1,1.5,0.1,Iris-setosa
9,5.4,3.7,1.5,0.2,Iris-setosa


In [19]:
## Discrete Feature Discrete Output Decision Tree

def entropy(target):
    freq = {}
    for elem in target:
        if elem not in freq:
            freq[elem]=1/len(target)
        else: 
            freq[elem]+=1/len(target)
    _sum = [-freq[i]*np.log2(freq[i]) for i in freq]
    return sum(_sum)

def info_gain(data,attributes):
    entropy_dataset = entropy(data['Class'])
    entr_val = {}
    infg = {}
    for attr in attributes:
        freq = {}
        for i in data[attr]:
            if i not in freq:
                freq[i]=1/len(data[attr])
            else:
                freq[i]+=1/len(data[attr])
        keys = list(freq.keys())
        entr = {}
        for i in keys:
            temp = len(data[data[attr]==i])
            entr[i] = temp/len(data)*entropy(data[data[attr]==i]['Class'])
        entr_val[attr] = sum(list(entr.values()))
        infg[attr] = entropy_dataset - entr_val[attr]
    return max(infg, key = infg.get)

def dtree(dataset,data,attributes,parent):
    attribute = [i for i in attributes]
    if len(data['Class'].unique())<=1:
        return (list(data['Class'].unique())[0])
    elif len(data)==0:
        return np.unique(data['Class'])[np.argmax(np.unique(data['Class'],return_counts=True)[1])]
    elif len(attribute)==0:
        return parent
    else:
        parent = np.unique(data['Class'])[np.argmax(np.unique(data['Class'],return_counts=True)[1])]
        
    best = info_gain(data,attribute)
    tree = {best:{}}
    attribute.remove(best)
    for ndata in data[best].unique():
        new_data = data[data[best]==ndata]
        subtree = dtree(dataset,new_data,attribute,parent)
        tree[best][ndata]=subtree
    return tree
    
    

In [ ]:
## Iris Dataset Classification


In [20]:
dt = pd.read_csv('tennis.csv')

In [21]:
dt = dt.drop(columns = ['Day'])

In [22]:
attrb = [i for i in dt.columns]
attrb.remove('Class')

In [24]:
tree = dtree(dt,dt,attrb,None)

In [25]:
tree

{'outlook': {'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'windy': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [39]:
def traverse(tree):
    for i in tree:
        if type(tree[i])==dict:
            traverse(tree[i])
        else:
            print(i,tree[i])

In [40]:
traverse(tree)

High No
Normal Yes
Overcast Yes
Weak Yes
Strong No
